In [1]:
import os
import shutil
import sys
from ultralytics import YOLO
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon

In [6]:
def copy_matching_jgw_files(jpg_folder, jgw_folder, output_dir):
    """
    Find .jgw files in jgw_folder that have the same base name as .jpg files in jpg_folder and copy
    them to output_dir.
    """
    # Get all .jpg files
    jpg_files = [f for f in os.listdir(jpg_folder) if f.lower().endswith('.jpg')]
    
    # Get all .jgw files
    jgw_files = [f for f in os.listdir(jgw_folder) if f.lower().endswith('.jgw')]
    
    # Get base names of jpg files (without extension)
    jpg_base_names = [os.path.splitext(f)[0] for f in jpg_files]
    
    # Keep track of matches
    matches_found = 0
    
    # Find matching .jgw files and make a copy them in jpg_folder
    for jgw_file in jgw_files:
        jgw_base_name = os.path.splitext(jgw_file)[0]
        
        if jgw_base_name in jpg_base_names:
            source_path = os.path.join(jgw_folder, jgw_file)
            dest_path = os.path.join(output_dir, jgw_file)
            
            try:
                shutil.copy2(source_path, dest_path)
                print(f"Copied: {jgw_file} to {dest_path}")
                matches_found += 1
            except Exception as e:
                print(f"Error copying {jgw_file}: {str(e)}")
    
    print(f"\nSummary: Copied {matches_found} .jgw files to {output_dir}")

def read_jgw(jgw_filepath):
    """ 
    Read the geographic information in a .jgw file and return it as a list.
    """
    with open(jgw_filepath, 'r') as file:
        lines = file.readlines()
        data = [float(line.strip()) for line in lines]
        return data

def transform_coordinates_numpy(xy, jgw_data, transform_func):
    """
    Transforms pixel coordinates in xy to geographic coordinates using the geographic data in
    jgw_data.
        
    Parameters:
    xy: List of numpy arrays with structure xy[polygon][point][coordinate]
    transform_func: Function that takes arrays of x, y and jgw_data as parameters and returns 
    new_x, new_y
    
    Returns:
    Transformed copy of the original structure
    """
    transformed = []
    
    # Iterate through each polygon
    for polygon in xy:
        # Convert to numpy array if not already
        poly_array = np.array(polygon)
        
        # Extract all x and y coordinates
        x_coords = poly_array[:, 0]
        y_coords = poly_array[:, 1]
        
        # Apply the transformation function
        new_x, new_y = transform_func(x_coords, y_coords, jgw_data)
        
        # Combine the transformed coordinates
        new_polygon = np.column_stack((new_x, new_y))
        transformed.append(new_polygon)
    
    return transformed

def transform(x, y, jgw_data):
    """ 
    Converts pixel coordinates to geographic coordinates by multiplying the x/y coordinate by the
    length of a pixel in the same direction and adding the respective x/y coordinate in the top left
    corner. 
    """
    new_x = jgw_data[4] + x * jgw_data[0]
    new_y = jgw_data[5] + y * jgw_data[3]
    return new_x, new_y

In [ ]:
# Define the path to the directory of images to make predictions on
imgs = "C:/Users/jdhoc/Desktop/DOT Volunteer Project/data/demo/dataset/segment/images/train"

# Run predictions on the images
model = YOLO("C:/Users/jdhoc/Desktop/DOT Volunteer Project/scripts/yolov11/runs/segment/train2/weights/best.pt")
results = model(imgs, stream = True)

output_dir = "C:/Users/jdhoc/Desktop/DOT Volunteer Project/data/demo/predictions"

# Copy the .jgw files that provide geographic information about the predicted images
copy_matching_jgw_files(imgs,
    r"C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\raster\output\NYC_ortho_2022\jpg",
    r"C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions")

predicted_xys = []
predicted_classes = []

# Process and save the results
for result in results:
    masks = result.masks  # Masks object for segmentation masks outputs
    xy = masks.xy
    print(xy) # xy[polygon][point][x or y]

    base_filename = os.path.basename(result.path)
    name, ext = os.path.splitext(base_filename)
    
    # Get the JGW data associated with the image to determine coordinate conversions
    jgw_data = read_jgw(f"C:/Users/jdhoc/Desktop/DOT Volunteer Project/data/demo/predictions/{name}.jgw")
    print(f"JGW Data for {name}: {jgw_data}")

    print(f"Class: {result.boxes.cls.cpu().numpy()}")

    # Convert to the pixels to geographic coordinates
    predicted_xys.append(transform_coordinates_numpy(xy, jgw_data, transform))

    # Save the classes of each prediction
    predicted_classes.append(result.boxes.cls.cpu().numpy())

    # Save the prediction image result
    result_filename = f"{name}{ext}"
    output_path = os.path.join(output_dir, result_filename)
    result.save(filename = output_path, boxes = False)

Copied: NYCortho22_66852.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_66852.JGw
Copied: NYCortho22_66853.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_66853.JGw
Copied: NYCortho22_66854.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_66854.JGw
Copied: NYCortho22_67155.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_67155.JGw
Copied: NYCortho22_67156.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_67156.JGw
Copied: NYCortho22_67157.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_67157.JGw
Copied: NYCortho22_67459.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_67459.JGw
Copied: NYCortho22_67460.JGw to C:\Users\jdhoc\Desktop\DOT Volunteer Project\data\demo\predictions\NYCortho22_67460.JGw

Summary: Copied 8 .jgw files to C:\User

In [10]:
def create_shapefile(predicted_xys, predicted_classes, output_path, crs="EPSG:6539"):
    """
    Create a single shapefile from multiple sets of polygon coordinates with roof class information.
    
    Parameters:
    predicted_xys: List where each item is a collection of polygons with the structure [i][polygon][point][x/y coordinate]
    predicted_classes: List of numpy arrays containing class values (0.0 for warm_roof, 1.0 for cool_roof)
    output_path: Path where the shapefile will be saved
    crs: Coordinate reference system (default: EPSG:6539)
        
    Returns:
    GeoDataFrame containing all polygons
    """
    all_polygons = []
    all_attributes = []
    
    # Process each instance
    for i, polygon_set in enumerate(predicted_xys):
        # Process each polygon in this instance
        for p, polygon_coords in enumerate(polygon_set):
            # Create Shapely polygon
            poly = Polygon(polygon_coords)
            
            # Get the class value for this polygon
            class_val = predicted_classes[i][p]
                
            all_polygons.append(poly)
            
            # Generate attributes for this polygon, including class information
            all_attributes.append({
                "instance_id": i, 
                "polygon_id": p,
                "cool_roof": int(class_val),  # 0 for warm_roof, 1 for cool_roof
                "roof_type": "cool_roof" if class_val == 1 else "warm_roof"
            })
    
    # Create a GeoDataFrame with all polygons
    gdf = gpd.GeoDataFrame(all_attributes, geometry=all_polygons, crs=crs)
    
    # Save to shapefile
    gdf.to_file(output_path)
    
    return gdf

In [11]:
attributes = []
output_path = "C:/Users/jdhoc/Desktop/DOT Volunteer Project/data/demo/predictions/predictions.shp"
gdf = create_shapefile(predicted_xys, predicted_classes, output_path)

C:\Users\jdhoc\AppData\Local\Temp\ipykernel_20924\3289490192.py:41: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_path)
c:\Users\jdhoc\Desktop\DOT Volunteer Project\scripts\cool_roofs_detection-venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'instance_id' to 'instance_i'
  ogr_write(
